In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, Input
from sklearn.preprocessing import MinMaxScaler

Sequential = tf.keras.models.Sequential
Adam = tf.keras.optimizers.Adam

df = pd.read_csv("sensor_data.csv", parse_dates=["timestamp"]).set_index("timestamp")
df = df[["temperature", "pressure", "humidity", "iaq"]].resample("5min").mean()
df = df.interpolate(method="time")

time_steps = 1000

data = np.column_stack([df["temperature"], df["humidity"], df["pressure"], df["iaq"]])

scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

def create_sequences(data, window_size):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i+window_size, :])
        y.append(data[i+window_size, :])
    return np.array(X), np.array(y)

window_size = 12  # last 5*12 minutes
X, y = create_sequences(data_scaled, window_size)

split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

model = Sequential([
    Input(shape=(window_size, 4)),
    # unroll=True for tflite conversion
    LSTM(64, return_sequences=True, unroll=True),
    LSTM(32, unroll=True),
    Dense(4)  # predicting 4 variables
])

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss="mse"
)

model.summary()

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=30,
    batch_size=32
)

#pred_scaled = model.predict(X_test)

# Inverse scale temperature only
#temp_min = scaler.data_min_[0]
#temp_max = scaler.data_max_[0]
#pred_temp = pred_scaled[:, 0] * (temp_max - temp_min) + temp_min
#true_temp = y_test[:, 0] * (temp_max - temp_min) + temp_min



Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 12, 64)         │        17,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4)              │           132 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 30,212 (118.02 KB)

 Trainable params: 30,212 (118.02 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 21s 718ms/step - loss: 0.2429 - val_loss: 0.0595
Epoch 2/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0418 - val_loss: 0.0454
Epoch 3/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0191 - val_loss: 0.0193
Epoch 4/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0123 - val_loss: 0.0134
Epoch 5/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0097 - val_loss: 0.0106
Epoch 6/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0078 - val_loss: 0.0092
Epoch 7/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0076 - val_loss: 0.0081
Epoch 8/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0074 - val_loss: 0.0078
Epoch 9/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0073 - val_loss: 0.0085
Epoch 10/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0076 - val_loss: 0.0078
Epoch 11/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0070 - val_loss: 0.0068
Epoch 12/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0066 - val

In [5]:
print(scaler.data_min_)
print(scaler.data_max_)

[14.92  33.38   1.005 50.   ]
[ 34.06  79.93   1.01 215.  ]


In [6]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
#converter.target_spec.supported_ops = [
#    tf.lite.OpsSet.TFLITE_BUILTINS,
#    tf.lite.OpsSet.SELECT_TF_OPS  # <-- IMPORTANT
#]
#converter._experimental_lower_tensor_list_ops = False
#converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

with open("model.tflite", "wb") as f:
    f.write(tflite_model)


Saved artifact at '/tmp/tmpq9u0rxb2'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 12, 4), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)
Captures:
  132742947239696: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132742947240656: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132742947241616: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132742947241808: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132742947240848: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132742947240464: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132742947238352: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132742947239120: TensorSpec(shape=(), dtype=tf.resource, name=None)
